## Perturbative footprint study: footprints clustering

### Imports

In [ ]:
# Import standard modules
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import MeanShift, estimate_bandwidth

# Nicer style
import seaborn as sns
sns.set_theme(style="whitegrid")
%config InlineBackend.figure_format='retina'


### Load footprints and plot a sample

In [ ]:
# Load l_fp
with open("output/l_fp.pkl", "rb") as f:
    l_fp = pickle.load(f)

# Load list of bunch numbers
with open("output/l_bunch_nb.pkl", "rb") as f:
    l_bunch_nb = pickle.load(f)

# Remove all nan footprints, along with bunch numbers
#l_bunch_nb = [bunch_nb for i, bunch_nb in enumerate(l_bunch_nb) if not np.isnan(l_fp[i]).any()]
#l_fp = [fp for fp in l_fp if not np.isnan(fp).any()]


# Make a grid of plots for all the footprints
sample_nb = 40
n_cols = 10
n_rows = int(np.ceil(len(l_fp[:sample_nb])/n_cols))
fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols*2.5, n_rows*2))
for i, fp in enumerate(l_fp[:sample_nb]):
    try:
        ax = axs[i//n_cols, i%n_cols]
        ax.plot(fp[0], fp[1], color="C0")
        ax.plot(fp[0].T, fp[1].T, color="C0")
        ax.set_title(f"Bunch {l_bunch_nb[i]}")
        ax.set_xlabel(r"$\mathrm{Q_x}$")
        ax.set_ylabel(r"$\mathrm{Q_y}$")
        #ax.set_aspect("equal")
        ax.set_title(f"Bunch {l_bunch_nb[i]}")
        ax.grid()
    except:
        pass

plt.tight_layout()
plt.show()

### Remove outliers in each figure

In [ ]:
# Look for outlier index in Qy distribution
l_qy = [fp[1] for fp in l_fp]
for i, qy in enumerate(l_qy):
    if np.min(qy) < 0.29:
        # Get outlier coordinates in 2D
        arg_outlier_flat = np.argmin(qy)
        print(f"Outlier found at index {i} (bunch {l_bunch_nb[i]}).")
        if i>80:
            break

# Clearly, there's a recurring outlier. Create a copy of fp where it's been replaced by nan
l_fp_no_outlier = [fp.copy() for fp in l_fp]
for i, fp in enumerate(l_fp_no_outlier):
    l_fp_no_outlier[i][0].flat[arg_outlier_flat] = np.nan
    l_fp_no_outlier[i][1].flat[arg_outlier_flat] = np.nan




### Get DA for each footprint and plot

In [ ]:
# Load dataframe for DA
path = "/afs/cern.ch/work/c/cdroin/private/DA_study/master_study/opt_flathv_75_1500_withBB_chroma15_1p4_all_bunches"
df = pd.read_parquet(f"{path}/da.parquet")

# Round all numbers to 3 decimals
df = df.round(3)

# Get list of bunches and list of DA
l_bunch_nb_from_DA = df["bunch_nb"]
l_DA = df["normalized amplitude in xy-plane"]
dic_DA = {bunch_nb: DA for bunch_nb, DA in zip(l_bunch_nb_from_DA, l_DA)}

In [ ]:
# Check that the bunch numbers are the same
assert all(l_bunch_nb_from_DA) == all(l_bunch_nb)

In [ ]:
# Plot DA vs bunch number
plt.figure(figsize=(10, 5))
plt.scatter(l_bunch_nb, [dic_DA[bunch_nb] for bunch_nb in l_bunch_nb], s = 2)
plt.xlabel("Bunch number")
plt.ylabel("DA")
plt.title("Evolution of the DA with bunch number")
plt.savefig("PDF_output/DA_vs_bunch_nb.pdf")
plt.show()

In [ ]:
# Make a grid of plots for all the footprints
n_cols = 10
n_rows = int(np.ceil(len(l_fp_no_outlier[:sample_nb])/n_cols))
fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols*2.5, n_rows*2))

# Build a discrete colormap for DAs
min_DA = np.min(l_DA)
max_DA = np.max(l_DA)
discrete_cmap = sns.color_palette("Spectral", n_colors=256)
l_colors = [discrete_cmap[int((DA-min_DA)/(max_DA-min_DA)*255)] for DA in l_DA]
dic_colors = {bunch_nb: color for bunch_nb, color in zip(l_bunch_nb_from_DA, l_colors)}

for i, (bunch_nb, fp) in enumerate(zip(l_bunch_nb[:sample_nb], l_fp_no_outlier[:sample_nb])):
    try:
        ax = axs[i//n_cols, i%n_cols]
        ax.plot(fp[0], fp[1], color=dic_colors[l_bunch_nb[i]])
        ax.plot(fp[0].T, fp[1].T, color=dic_colors[l_bunch_nb[i]])
        ax.set_title(f"Bunch {l_bunch_nb[i]}, DA={dic_DA[l_bunch_nb[i]]:.3f}")
        ax.set_xlabel(r"$\mathrm{Q_x}$")
        ax.set_ylabel(r"$\mathrm{Q_y}$")
        #ax.set_aspect("equal")
        ax.set_xlim(np.min(fp[0])-0.01, np.max(fp[0])+0.01)
        ax.set_ylim(np.min(fp[1])-0.01, np.max(fp[1])+0.01)
        ax.set_title(f"Bunch {l_bunch_nb[i]}")
        ax.grid()
    except:
        pass

plt.tight_layout()
plt.show()

### Reduce dimensionality with PCA

In [ ]:
# Convert matrix samples to a row
l_fp_flat = []
for i, fp in enumerate(l_fp):
    # Process qx and qy separately
    fp_x = fp[0].flatten()
    fp_y = fp[1].flatten()

    # Remove outlier
    fp_x_deleted = np.delete(fp_x, arg_outlier_flat)
    fp_y_deleted = np.delete(fp_y, arg_outlier_flat)
    # Replace by nan
    #fp_x_deleted = np.insert(fp_x_deleted, arg_outlier_flat, 0.3)
    #fp_y_deleted = np.insert(fp_y_deleted, arg_outlier_flat, 0.3)
    
    l_fp_flat.append([fp_x_deleted, fp_y_deleted])
    #l_fp_flat.append([fp_x, fp_y])
array_fp_flat = np.array(l_fp_flat)

# apply tsne to the data: rebuild the 2D matrix with only 1 components for each tune dimension
pca = PCA(n_components=5)
#pca = umap.UMAP(random_state=42)

array_fp_flat_qx_ld = pca.fit_transform(array_fp_flat[:,0,:])
array_fp_flat_qx_compressed = pca.inverse_transform(array_fp_flat_qx_ld)
array_fp_flat_qy_ld = pca.fit_transform(array_fp_flat[:,1,:])
array_fp_flat_qy_compressed = pca.inverse_transform(array_fp_flat_qy_ld)

# Insert np.nan for outlier and reshape compressed arrays to original shape
array_fp_flat_qx_compressed = np.insert(array_fp_flat_qx_compressed, arg_outlier_flat, np.nan, axis = -1)
array_fp_flat_qy_compressed = np.insert(array_fp_flat_qy_compressed, arg_outlier_flat, np.nan, axis = -1)
array_fp_flat_qx_compressed = np.reshape(array_fp_flat_qx_compressed, (array_fp_flat_qx_compressed.shape[0], l_fp[0][0].shape[0], l_fp[0][0].shape[1]))
array_fp_flat_qy_compressed = np.reshape(array_fp_flat_qy_compressed, (array_fp_flat_qy_compressed.shape[0], l_fp[0][1].shape[0], l_fp[0][1].shape[1]))
array_fp_flat_compressed = np.array([array_fp_flat_qx_compressed, array_fp_flat_qy_compressed])

# switch dimension 0 and 1
array_fp_compressed = np.swapaxes(array_fp_flat_compressed, 0, 1)

print(array_fp_compressed.shape)

In [ ]:
# Make a grid of plots for all the footprints
n_cols = 10
n_rows = int(np.ceil(len(l_fp_no_outlier[:sample_nb])/n_cols))
fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols*2.5, n_rows*2))
for i, (fp_compressed, fp_original) in enumerate(zip(array_fp_compressed[:sample_nb], l_fp_no_outlier[:sample_nb])):
    try:
        ax = axs[i//n_cols, i%n_cols]
        ax.plot(fp_compressed[0], fp_compressed[1], color="C0")
        ax.plot(fp_compressed[0].T, fp_compressed[1].T, color="C0")
        ax.plot(fp_original[0], fp_original[1], color="C1", alpha = 0.5)
        ax.plot(fp_original[0].T, fp_original[1].T, color="C1", alpha = 0.5)
        ax.set_title(f"Bunch {l_bunch_nb[i]}")
        ax.set_xlabel(r"$\mathrm{Q_x}$")
        ax.set_ylabel(r"$\mathrm{Q_y}$")
        #ax.set_aspect("equal")
        ax.set_xlim(np.min(fp_original[0])-0.01, np.max(fp_original[0])+0.01)
        ax.set_ylim(np.min(fp_original[1])-0.01, np.max(fp_original[1])+0.01)
        ax.set_title(f"Bunch {l_bunch_nb[i]}")
        ax.grid()
    except:
        pass

plt.tight_layout()
plt.savefig("output/footprints_compressed.png")
#plt.show()

### Cluster the resulting points

In [ ]:
# Get 4D data
array_fp_ld = np.hstack([array_fp_flat_qx_ld, array_fp_flat_qy_ld])

# Apply HDBSCAN
#clusterer = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=10).fit(array_fp_ld)
#
bandwidth = estimate_bandwidth(array_fp_ld, quantile=0.13, n_samples=len(array_fp_ld))
clusterer = MeanShift(bandwidth=bandwidth, bin_seeding=True).fit(array_fp_ld)
clusterer.labels_


In [ ]:
# Plot initial data according to labels
n_cols = 10
n_rows = int(np.ceil(len(l_fp[:sample_nb])/n_cols))
fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols*2.5, n_rows*2))
for i, fp in enumerate(l_fp[:sample_nb]):
    try:
        ax = axs[i//n_cols, i%n_cols]
        ax.plot(fp[0], fp[1], color="C" + str(clusterer.labels_[i]))
        ax.plot(fp[0].T, fp[1].T, color="C" + str(clusterer.labels_[i]))
        ax.set_title(f"Bunch {l_bunch_nb[i]}")
        ax.set_xlabel(r"$\mathrm{Q_x}$")
        ax.set_ylabel(r"$\mathrm{Q_y}$")
        #ax.set_aspect("equal")
        ax.set_title(f"Bunch {l_bunch_nb[i]}")
        ax.grid()
    except:
        pass

plt.tight_layout()
plt.show()

### Plot DA distribution per cluster

In [ ]:
# Compute a dataframe of DA for each cluster

dic_cluster_DA = {f'C$_{{{i}}}$':[] for i in np.unique(clusterer.labels_)}
for i, bunch_nb in enumerate(l_bunch_nb):
    # Get cluster number
    cluster_nb = clusterer.labels_[i]
    dic_cluster_DA[f'C$_{{{cluster_nb}}}$'].append(dic_DA[bunch_nb])

# Get list of cluster numbers and list of DA for each cluster
l_cluster_nb = list(dic_cluster_DA.keys())
l_DA_violin = list(dic_cluster_DA.values())

# Sort by min DA in l_DA
l_DA_sorted = [x for _,x in sorted(zip([np.min(x) for x in l_DA_violin], l_DA_violin))]
l_cluster_nb_sorted = [x for _,x in sorted(zip([np.min(x) for x in l_DA_violin], l_cluster_nb))]

fig, ax = plt.subplots(figsize=(9, 6))

def set_axis_style(ax, labels):
    ax.set_xticks(np.arange(1, len(labels) + 1), labels=labels)
    ax.set_xlim(0.25, len(labels) + 0.75)
    ax.set_xlabel('Cluster')

plt.violinplot(dataset=l_DA_sorted )
# set style for the axes
set_axis_style(ax, l_cluster_nb_sorted)
plt.ylabel("DA (sigma)")
plt.title("Distribution of DA for each cluster")
plt.savefig("PDF_output/DA_distribution.pdf")
plt.show()


### Build PDF files to represent all footprints

In [ ]:
# Build a file where footprints are colored by DA
# Sample one bunch out of 10 to keep figure readable

#Make a grid of plots for all the footprints
n_cols = 10
n_rows = int(np.ceil(len(l_fp_no_outlier[::10])/n_cols))
fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols*2.5, n_rows*2))
#plt.suptitle("Footprints colored by DA")

# Build a discrete colormap for DAs
min_DA = np.min(l_DA)
max_DA = np.max(l_DA)
discrete_cmap = sns.color_palette("Spectral", n_colors=256)
l_colors = [discrete_cmap[int((DA-min_DA)/(max_DA-min_DA)*255)] for DA in l_DA]
dic_colors = {int(bunch_nb): color for bunch_nb, color in zip(l_bunch_nb_from_DA, l_colors)}


# # Get ax limits
min_x = np.min([np.nanmin(fp[0]) for fp in l_fp_no_outlier])
max_x = np.max([np.nanmax(fp[0]) for fp in l_fp_no_outlier])
min_y = np.min([np.nanmin(fp[1]) for fp in l_fp_no_outlier])
max_y = np.max([np.nanmax(fp[1]) for fp in l_fp_no_outlier])

for i, (bunch_nb, fp) in enumerate(zip(l_bunch_nb[::10], l_fp_no_outlier[::10])):
    ax = axs[i//n_cols, i%n_cols]
    ax.plot(fp[0], fp[1], color=dic_colors[bunch_nb])
    ax.plot(fp[0].T, fp[1].T, color=dic_colors[bunch_nb])
    ax.set_title(f"Bunch {bunch_nb}, DA={dic_DA[bunch_nb]:.3f}, cluster {clusterer.labels_[::10][i]}", fontsize=8)
    ax.set_xlabel(r"$\mathrm{Q_x}$", fontsize=6)
    ax.set_ylabel(r"$\mathrm{Q_y}$", fontsize=6)
    # Set smaller ticklabels
    ax.tick_params(axis='both', which='major', labelsize=6)
    ax.tick_params(axis='both', which='minor', labelsize=6)
    #ax.set_aspect("equal")
    ax.set_xlim(min_x-0.001, max_x+0.001)
    ax.set_ylim(min_y-0.001, max_y+0.001)
    #ax.set_title(f"Bunch {l_bunch_nb[i]}, DA={dic_DA[l_bunch_nb[i]]:.3f}, cluster {clusterer.labels_[i]}")
    ax.grid()

plt.tight_layout()
plt.savefig("PDF_output/footprints_colored_by_DA.pdf")
#plt.show()

In [ ]:
# Build a file where footprints are colored by cluster
# Sample one bunch out of 10 to keep figure readable

#Make a grid of plots for all the footprints
n_cols = 10
n_rows = int(np.ceil(len(l_fp_no_outlier[::10])/n_cols))
fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols*2.5, n_rows*2))
#plt.suptitle("Footprints colored by cluster")


# # Get ax limits
min_x = np.min([np.nanmin(fp[0]) for fp in l_fp_no_outlier])
max_x = np.max([np.nanmax(fp[0]) for fp in l_fp_no_outlier])
min_y = np.min([np.nanmin(fp[1]) for fp in l_fp_no_outlier])
max_y = np.max([np.nanmax(fp[1]) for fp in l_fp_no_outlier])

for i, (bunch_nb, fp) in enumerate(zip(l_bunch_nb[::10], l_fp_no_outlier[::10])):
    ax = axs[i//n_cols, i%n_cols]
    ax.plot(fp[0], fp[1],  color="C" + str(clusterer.labels_[i]))
    ax.plot(fp[0].T, fp[1].T, color="C" + str(clusterer.labels_[i]))
    ax.set_title(f"Bunch {bunch_nb}, DA={dic_DA[bunch_nb]:.3f}, cluster {clusterer.labels_[::10][i]}", fontsize=8)
    ax.set_xlabel(r"$\mathrm{Q_x}$", fontsize=6)
    ax.set_ylabel(r"$\mathrm{Q_y}$", fontsize=6)
    # Set smaller ticklabels
    ax.tick_params(axis='both', which='major', labelsize=6)
    ax.tick_params(axis='both', which='minor', labelsize=6)
    #ax.set_aspect("equal")
    ax.set_xlim(min_x-0.001, max_x+0.001)
    ax.set_ylim(min_y-0.001, max_y+0.001)
    #ax.set_title(f"Bunch {l_bunch_nb[i]}, DA={dic_DA[l_bunch_nb[i]]:.3f}, cluster {clusterer.labels_[i]}")
    ax.grid()

plt.tight_layout()
plt.savefig("PDF_output/footprints_colored_by_cluster.pdf")
#plt.show()